## State-of-the-Art Sentiment Classification in TensorFlow

In the previous tutorial ([1-Construct-TensorFlow-Data-Pipeline.ipynb](https://github.com/ralphbrooks/tensorflow-tutorials/blob/master/1-Construct-TensorFlow-Data-Pipeline.ipynb)), I showed how you can create a data pipeline for a TensorFlow model. Once you have this data pipeline prepared, you are now ready to ingest this data and create a state-of-the-art classifier of emotion in language (a sentiment classifier). 

To this this we are going to make use of the following technologies:

1) <b>Transfer Learning:</b> In 2019, Transfer Learning showed that it could achieve good performance on relatively low amounts of data. The concept is that a model is trained on a large amount of data beforehand, and this "pre-trained" model is then used in a second stage for classification. 

2) [<b>Google BERT:</b>](https://ai.googleblog.com/2018/11/open-sourcing-bert-state-of-art-pre.html) This is model that was pretrained by Google off of information from Wikipedia. It attempts to predict a word in a sentence given the words that PRECEDE the missing word and the words that FOLLOW the missing word. The model combines this prediction task with a [Transformer model architecture](https://ai.googleblog.com/2017/08/transformer-novel-neural-network.html) which builds a neural network that looks at the emphasis of each word in relation to all other words in the sentence (a concept called self-attention).

3) [<b>Huggingface Transformers:</b>](https://github.com/huggingface/transformers) - A startup company called Huggingface related in 2019 "wrapper code" that simplies using BERT as part of a data pipeline. 




### Step 1: Import and Setup

We will get started by importing in the GPU version of Tensorflow and the Transformers library from Huggingface.

Notes:

* This exercise is going to require running this code with a GPU. I ran this code in AWS Sagemaker using a ml.p3.2xlarge GPU. Keep in mind that this GPU (as of Nov. 2019) costs a little
over $4 / hour to operate.

* Said differently, make sure that you stop or terminate your AWS notebook after completing this example so that you do not incur unnecessary cloud compute fees.



In [1]:
%load_ext autoreload
% autoreload 2

In [2]:
!pip install --upgrade pip

    100% |████████████████████████████████| 1.4MB 22.6MB/s ta 0:00:01
  Found existing installation: pip 10.0.1
    Uninstalling pip-10.0.1:
      Successfully uninstalled pip-10.0.1


Since this exercise involves training a model, it is important that you use the GPU version of tensorflow for the training.

In [3]:
!yes | pip uninstall tensorflow
! pip install tensorflow-gpu==2.0.0

Uninstalling tensorflow-1.14.0:
  Would remove:
    /home/ec2-user/anaconda3/envs/tensorflow_p36/bin/freeze_graph
    /home/ec2-user/anaconda3/envs/tensorflow_p36/bin/saved_model_cli
    /home/ec2-user/anaconda3/envs/tensorflow_p36/bin/tensorboard
    /home/ec2-user/anaconda3/envs/tensorflow_p36/bin/tf_upgrade_v2
    /home/ec2-user/anaconda3/envs/tensorflow_p36/bin/tflite_convert
    /home/ec2-user/anaconda3/envs/tensorflow_p36/bin/toco
    /home/ec2-user/anaconda3/envs/tensorflow_p36/bin/toco_from_protos
    /home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/tensorflow-1.14.0.dist-info/*
    /home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/tensorflow/*
Proceed (y/n)?   Successfully uninstalled tensorflow-1.14.0
yes: standard output: Broken pipe
yes: write error
     |████████████████████████████████| 380.8MB 26kB/s s eta 0:00:01     |██████████████████▌             | 220.4MB 37.8MB/s eta 0:00:05     |██████████████████████▉         | 271.0

In [4]:
!pip install -q transformers==2.1.1

In [5]:
import tensorflow as tf
from transformers import *
from transformers import BertTokenizer, TFBertForSequenceClassification, glue_convert_examples_to_features


I1111 21:42:32.544695 139735363929920 file_utils.py:32] TensorFlow version 2.0.0 available.


In [6]:
tf.__version__

'2.0.0'

### Step 2: Create tf.data.Dataset from TFRecord

In the previous tutorial, we had put together the encoded data as TFRecords. Now, we will set up a transformation pipeline with a ```tf.data.Dataset``` .

A ```tf.data.Dataset``` allows you to construct the pipeline without having to ingest and process all of the data in each step before examining the next transformation step. This is massively critical if you are testing out your pipeline. 

Said differently, the last thing in the world that you want to do is to wait 3-4 minutes during an iteration of code because you are waiting for a stage of preprocessing to complete. 

&nbsp;  
    
The following code extracts the data that will be used to train the model (the tr_ds or train dataset), the data that will be used to validate the accuracy of the model (val_ds) and the holdout set of data that is used for the final verification of model accuracy.

In [7]:
tr_ds = tf.data.TFRecordDataset("data/yelp_train.tfrecord")
val_ds = tf.data.TFRecordDataset("data/yelp_validate.tfrecord")
test_ds = tf.data.TFRecordDataset("data/yelp_test.tfrecord")

The following code will now extract tensors (the basic building blocks in TensorFlow) from the TFRecord. In essence, these tensors have the potential to be used as inputs into the model. 

Notes: 

* When you are extracting features from the TFRecord Dataset, it is important to note that <b>the keys in the feature spec must match the keys that that were used in the TFRecord encoding process </b> (the process that was covered in the previous tutorial). Also note that you don't have to use all of the tensors that are encoded within TFRecord (you could use just a subset of the Tensors). 

In [8]:
# convert the encoded string tensor into the separate tensors that will feed into the model

feature_spec = {
    'idx': tf.io.FixedLenFeature([], tf.int64),
    'sentence': tf.io.FixedLenFeature([], tf.string),
    'label': tf.io.FixedLenFeature([], tf.int64)
}

def parse_example(example_proto):
  # Parse the input tf.Example proto using the dictionary above.
  return tf.io.parse_single_example(example_proto, feature_spec)

tr_parse_ds = tr_ds.map(parse_example)
val_parse_ds = val_ds.map(parse_example)
test_parse_ds = test_ds.map(parse_example)

One approach to cleaning up a pipeline is to map a function to the dataset. In this way, the function gets applied to each example. The following code uses this approach to clean up the sentence tensor.

In [9]:

def clean_yelp_string(features):
    revised_sentence = tf.strings.regex_replace(features['sentence'], "\.\.\.", "", replace_global=True)
    revised_sentence = tf.strings.regex_replace(revised_sentence, "\\'", "'", replace_global=True)
    revised_sentence = tf.strings.regex_replace(revised_sentence, "\\n", "", replace_global=True)
    features['sentence'] = revised_sentence
    return features

In [10]:
tr_clean_ds = tr_parse_ds.map(lambda features: clean_yelp_string(features))
val_clean_ds = val_parse_ds.map(lambda features: clean_yelp_string(features))
test_clean_ds = test_parse_ds.map(lambda features: clean_yelp_string(features))

### Step 3: Train the model

Recent research( [https://arxiv.org/abs/1804.07612](https://arxiv.org/abs/1804.07612)) suggests that smaller batch training improves generalization. Based on this, I set the batch size for training at 32. 

In [11]:
BATCH_SIZE = 32

EVAL_BATCH_SIZE = BATCH_SIZE * 2

# XLA is the optimizing compiler for machine learning
# It can potentially increase speed by 15% with no source code changes
USE_XLA = False

# mixed precision results on https://github.com/huggingface/transformers/tree/master/examples
# Mixed precision can help to speed up training time
USE_AMP = False

Now you need to pull in the lengths of the different datasets from the JSON file that we created in the last tutorial. 

In [12]:
# Steps is determined by the number of examples
import json

with open('data/yelp_info.json') as json_file:
    data_info = json.load(json_file)
    
train_examples = data_info['train_length']
valid_examples = data_info['validation_length']
test_examples = data_info['test_length']

train_examples, valid_examples, test_examples

(738, 82, 82)

In [13]:
tf.config.optimizer.set_jit(USE_XLA)
tf.config.optimizer.set_experimental_options({"auto_mixed_precision": USE_AMP})

Now that we have a pipeline setup, we need to start the process of converting words into numbers so that they can be processed by the BERT transfer learning backbone. This process is commonly called Tokenization, and Huggingface includes a tokenizer that helps with this process.

The tokenizers are based on the underlying research code. For example, the following are different BERT models that can be utilized within the BERT framework:

    * ``bert-base-uncased``: 12-layer, 768-hidden, 12-heads, 110M parameters
    * ``bert-large-uncased``: 24-layer, 1024-hidden, 16-heads, 340M parameters
    * ``bert-base-cased``: 12-layer, 768-hidden, 12-heads , 110M parameters
    * ``bert-large-cased``: 24-layer, 1024-hidden, 16-heads, 340M parameters
    
As seen above, the different numbers have different levels of complexity and are associated either with uncapitalized text (uncased) or text that has capitalization. I selected the bert-base-cased underlying model because the Yelp utterances have capitalization. I also selected it because in general models that are less complex tend to run faster than models which are more complex.


In [14]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

I1111 21:42:53.423637 139735363929920 file_utils.py:296] https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-cased-vocab.txt not found in cache or force_download set to True, downloading to /tmp/tmpaoidsjxi
100%|██████████| 213450/213450 [00:00<00:00, 978875.01B/s]
I1111 21:42:53.970472 139735363929920 file_utils.py:309] copying /tmp/tmpaoidsjxi to cache at /home/ec2-user/.cache/torch/transformers/5e8a2b4893d13790ed4150ca1906be5f7a03d6c4ddf62296c383f6db42814db2.e13dbb970cb325137104fb2e5f36fe865f27746c6b526f6352861b1980eb80b1
I1111 21:42:53.972203 139735363929920 file_utils.py:313] creating metadata file for /home/ec2-user/.cache/torch/transformers/5e8a2b4893d13790ed4150ca1906be5f7a03d6c4ddf62296c383f6db42814db2.e13dbb970cb325137104fb2e5f36fe865f27746c6b526f6352861b1980eb80b1
I1111 21:42:53.973246 139735363929920 file_utils.py:322] removing temp file /tmp/tmpaoidsjxi
I1111 21:42:53.973972 139735363929920 tokenization_utils.py:374] loading file https://s3.amazonaws.com/models.h

The Transformers framework can use a configuration dictionary in order to set up the hyperparameters for the model. In this case, I explictly use the config to make sure that the model
is looking at num_labels=2. If we had been going through an example with three categories ('Positive', 'Negative', and 'Neutral') as opposed to just two cases ('Positive' and 'Negative') then we would have wanted to use num_labels=3 instead.

In [15]:
# Make use of the following config parameters which are stored in bert_config.json.

# {
#   "attention_probs_dropout_prob": 0.1,
#   "hidden_act": "gelu",
#   "hidden_dropout_prob": 0.1,
#   "hidden_size": 768,
#   "initializer_range": 0.02,
#   "intermediate_size": 3072,
#   "layer_norm_eps": 1e-12,
#   "max_position_embeddings": 512,
#   "num_attention_heads": 12,
#   "num_hidden_layers": 12,
#   "num_labels": 2,
#   "output_attentions": false,
#   "output_hidden_states": false,
#   "output_past": true,
#   "pruned_heads": {},
#   "torchscript": false,
#   "type_vocab_size": 2,
#   "use_bfloat16": false,
#   "vocab_size": 28996
# }

In [16]:
from transformers.configuration_bert import BertConfig
config = BertConfig("bert_config.json")

TensorFlow uses layers of abstraction when putting together a model. Operations (such as matrix algebra) can occur at a low level, and an abstraction of a neural network layer can occur at a higher level. One of these higher levels of abstraction is called a Keras model, and Huggingface uses this model as a way to abstract some of the granular details involved in using BERT for transfer learning.

The following is the syntax to instantiate a keras model with the Huggingface framework. 

In [17]:
model = TFBertForSequenceClassification.from_pretrained('bert-base-cased', config=config)

I1111 21:42:54.523327 139735363929920 file_utils.py:296] https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-cased-tf_model.h5 not found in cache or force_download set to True, downloading to /tmp/tmpprqoym6o
100%|██████████| 526681800/526681800 [00:23<00:00, 22244750.33B/s]
I1111 21:43:18.652600 139735363929920 file_utils.py:309] copying /tmp/tmpprqoym6o to cache at /home/ec2-user/.cache/torch/transformers/54c7ddffbd4d74d7592fac0cca93a6542cfee71482507625c277ddd4c6b7d41c.908e74db1113031d6827eb22808cf370b0aeded6e6ac20d0f07af0a334e195cc.h5
I1111 21:43:19.183894 139735363929920 file_utils.py:313] creating metadata file for /home/ec2-user/.cache/torch/transformers/54c7ddffbd4d74d7592fac0cca93a6542cfee71482507625c277ddd4c6b7d41c.908e74db1113031d6827eb22808cf370b0aeded6e6ac20d0f07af0a334e195cc.h5
I1111 21:43:19.185171 139735363929920 file_utils.py:322] removing temp file /tmp/tmpprqoym6o
I1111 21:43:19.265163 139735363929920 modeling_tf_utils.py:258] loading weights file https://s3


Now that we have a tokenizer and the model with the right configurations, we need to take our parsed tensors (the tr_parse_ds or train parsed dataset) and feed them into the Huggingface framework. To do this, we are going to make a slight modification to the glue_convert_examples_to_features code found in the
HuggingFace transformers repo. Here we are going to use the sst-2 task (the Stanford Sentiment Treebank binary classification task) because this task also works with binary classification. Because our example uses '1' for 'Negative' and '3' for Positive, we need to make this explicit by including the label_list keyword when doing the conversion. 


Notes:
* Huggingface uses the similar strategy of taking the TFExamples and using a dataset in order to convert the "sentence" and "labels" into inputs that are needed by BERT (inputs such as 'input_ids', 'attention_mask', and 'token_type_ids'). As disscussed earlier in the workbook, this transformation process makes it quick to test out the conversion on a couple of data points and to move onto the next step without waiting for the full conversion to complete. 

In [18]:
train_dataset = glue_convert_examples_to_features(examples=tr_clean_ds, tokenizer=tokenizer
                                                  , max_length=128, task='sst-2'
                                                  , label_list =['1', '3'])

I1111 21:43:26.696814 139735363929920 glue.py:73] Using output mode classification for task sst-2
I1111 21:43:26.807705 139735363929920 glue.py:80] Writing example 0
I1111 21:43:26.809409 139735363929920 glue.py:119] *** Example ***
I1111 21:43:26.810100 139735363929920 glue.py:120] guid: 367
I1111 21:43:26.810755 139735363929920 glue.py:121] input_ids: 101 1188 1110 170 6431 7054 1708 2984 1115 4252 18389 1116 1107 8132 1555 106 1109 2618 117 1155 1103 1236 1205 1106 1103 2546 1132 1632 1120 15196 5793 119 8774 2490 1107 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
I1111 21:43:26.811455 139735363929920 glue.py:122] attention_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

Repeate the same transformation that you did for the training set on the information that will be used to validate the performance of the model. 

In [19]:
valid_dataset = glue_convert_examples_to_features(examples=val_clean_ds, tokenizer=tokenizer
                                                  , max_length=128, task='sst-2'
                                                  , label_list =['1', '3'])

I1111 21:43:27.591511 139735363929920 glue.py:73] Using output mode classification for task sst-2
I1111 21:43:27.688185 139735363929920 glue.py:80] Writing example 0
I1111 21:43:27.689889 139735363929920 glue.py:119] *** Example ***
I1111 21:43:27.690478 139735363929920 glue.py:120] guid: 16
I1111 21:43:27.691060 139735363929920 glue.py:121] input_ids: 101 142 3190 10954 23955 11680 1942 8132 4125 119 17244 5578 117 1589 1304 1662 1106 2222 1106 10820 1412 2492 1114 141 18172 8231 1942 1794 119 3350 1110 1304 3044 1895 117 20122 2165 1105 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
I1111 21:43:27.691619 139735363929920 glue.py:122] attention_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

One of the advantage of using the dataset approach is that you can specify the size of your batches, and you can shuffle the data.

In [20]:
train_dataset = train_dataset.shuffle(train_examples).batch(BATCH_SIZE).repeat(-1)

valid_dataset = valid_dataset.batch(EVAL_BATCH_SIZE)

In this next section, we need to configure the loss function, the optimizer, and any additional metrics that we want to capture.

<b>Loss:</b> 

This is the objective that the model is trying to minimize. In our example, the HuggingFace framework converts our Negatives ('1') and our Positives ('3') into '0' and '1' and compares this against the distribution of predicted classes. Said differently, we are trying to compare how similar the actual distribution is to the predicted distribution, and this is captured in the loss function called ```SparseCategoricalCrossentropy``` . 

A good discussion on cross entropy can be found at [The Gradient](https://thegradient.pub/understanding-evaluation-metrics-for-language-models/)

<b>Optimizer:</b>

Deep learning is the process of minimizing a loss function. The process that determines what steps to try out in each iteration is commonly referred to as the optimizer. For this exercise, I used the [Adam optimization algorithm](https://arxiv.org/pdf/1412.6980.pdf) which tends to work well in a variety of situations. 

<b> Metric: </b> 

As a basic metric, we should look at the number of times that the actual class is identical  to what is predicted. 

Unfortunately ,the model currently generates unscaled outputs for each example (an unscaled output for the negative class and another unscaled output for the positive class). Said differently, the model generates outputs before they are converted into probabilities (the conversion happens with a softmax function). Because of all of this, the appropriate metric to use would be ```SparseCategoricalAccuracy```.

&nbsp;  

In [21]:
opt = tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08)

if USE_AMP:
    # loss scaling is currently required when using mixed precision
    opt = tf.keras.mixed_precision.experimental.LossScaleOptimizer(opt, 'dynamic')

loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
model.compile(optimizer=opt, loss=loss, metrics=[metric])

The TensorFlow documentation states the following:

<b>If x is a tf.data dataset, and 'steps_per_epoch' is None, the epoch will run until the input dataset is exhausted.</b>

Because these datasets can be a precursor to training on TFRecords of significant size, it is best practice to specificially state the number of steps that will be processed per epoch in the train and validation stage. 



In [22]:
train_steps = train_examples//BATCH_SIZE
valid_steps = valid_examples//EVAL_BATCH_SIZE

GPUs run up to 27x faster that CPUs for model training. Because of this, it is critical that the following preconditions are in place:

* You are using the version of Tensflow for GPUs
* The code has access to a GPU

To confirm the preconditions, I run the following code to detect GPUs and to see the physical devices that are available.

In [23]:
# GPU USAGE
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


In [24]:
tf.config.experimental.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:XLA_CPU:0', device_type='XLA_CPU'),
 PhysicalDevice(name='/physical_device:XLA_GPU:0', device_type='XLA_GPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [25]:
model.summary()

Model: "tf_bert_for_sequence_classification"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  108310272 
_________________________________________________________________
dropout_37 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  1538      
Total params: 108,311,810
Trainable params: 108,311,810
Non-trainable params: 0
_________________________________________________________________


In [26]:
history = model.fit(train_dataset, epochs=3, steps_per_epoch=train_steps,
                    validation_data=valid_dataset, validation_steps=valid_steps)

Train for 23 steps, validate for 1 steps
Epoch 1/3
23/23 [==============================] - 33s 1s/step - loss: 0.5825 - accuracy: 0.7418 - val_loss: 0.3751 - val_accuracy: 0.8750
Epoch 2/3
23/23 [==============================] - 7s 289ms/step - loss: 0.3176 - accuracy: 0.8810 - val_loss: 0.5090 - val_accuracy: 0.7344
Epoch 3/3
23/23 [==============================] - 7s 289ms/step - loss: 0.2096 - accuracy: 0.9320 - val_loss: 0.2417 - val_accuracy: 0.9062



The above output shows the following 

* In the first epoch, the train accuracy is 74% and the validation accuracy is 87.5%. This suggests that the model is underfitting. 

* In the second epoch, I start to see overfitting (and this carries into the third epoch). Normally, the solution to an overfitting model would be to add additional data, but the purpose of this notebook is to show high accuracy when there is a limited amount of training data. 


At this point, you see that the model has accuracy of 90%. This is fairly good considering that the Yelp rating captures the full sentiment of the user, but that in many cases, the Yelp API is only giving you part of the actual text review.

### Step 4: Evaluate the results of the model

The results above seem to look good. At this point, it is worth doing one final check before calling it a day.

In [27]:
test_dataset = glue_convert_examples_to_features(examples=test_clean_ds, tokenizer=tokenizer
                                                  , max_length=128, task='sst-2'
                                                  , label_list =['1', '3'])

I1107 17:40:46.695626 140320940463936 glue.py:73] Using output mode classification for task sst-2
I1107 17:40:46.712742 140320940463936 glue.py:80] Writing example 0
I1107 17:40:46.714177 140320940463936 glue.py:119] *** Example ***
I1107 17:40:46.714807 140320940463936 glue.py:120] guid: 293
I1107 17:40:46.715442 140320940463936 glue.py:121] input_ids: 101 146 1108 1189 1317 4437 4423 1103 107 20315 112 188 27502 107 1788 117 6142 1128 112 1231 1682 1106 2222 1149 1103 5197 1111 1489 1552 1196 18147 1106 170 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
I1107 17:40:46.716065 140320940463936 glue.py:122] attention_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

In [28]:
test_dataset = test_dataset.batch(EVAL_BATCH_SIZE)

In [29]:
model.evaluate(test_dataset)

2/2 [==============================] - 0s 191ms/step - loss: 0.3048 - accuracy: 0.8537


[0.3048120141029358, 0.85365856]

Here we see that the model has 84% accuracy in the test set. 

### Step 5: Create a Confusion Matrix

&nbsp;  

We can also visualize the evaluation in terms of how many true positives, true negatives, false positives, and false negatives occur. This visualization is commonly called a Confusion Matrix.

Creation of the confusion matrix involves the following steps:

1) Take the unnormalized outputs from the model (the logits) and compress them into probabilities (that by definition are between 0 and 1). The function that converts logits into probabilities is the softmax function.

2) Once you have probabilities for each prediction, identify predicted emotion ('Negative' or 'Positive') by selecting the probability with the largest value. This is accomplished with the argmax function.


In [30]:
y_pred = tf.nn.softmax(model.predict(test_dataset))

In [31]:
y_pred[:10]

<tf.Tensor: id=48755, shape=(10, 2), dtype=float32, numpy=
array([[0.9743283 , 0.02567171],
       [0.7790352 , 0.22096482],
       [0.02340363, 0.97659636],
       [0.09775554, 0.90224445],
       [0.963593  , 0.03640702],
       [0.7898381 , 0.21016194],
       [0.96113175, 0.03886823],
       [0.38373765, 0.6162624 ],
       [0.01492541, 0.98507464],
       [0.17768177, 0.8223182 ]], dtype=float32)>

In [32]:
y_pred_argmax = tf.math.argmax(y_pred, axis=1)

In [33]:
y_pred_argmax[:10]

<tf.Tensor: id=48761, shape=(10,), dtype=int64, numpy=array([0, 0, 1, 1, 0, 0, 0, 1, 1, 1])>

3) Use a simple for loop in order to pull the true values from the test dataset. 

Be aware though that the Huggingface framework converted the labels that we had ('1' for negative and '3' for positive) into '0' for negative and '1' for positive.

In [34]:
y_true = tf.Variable([], dtype=tf.int64)

for features, label in test_dataset.take(-1):
    y_true = tf.concat([y_true, label], 0)

In [35]:
y_true[:30]

<tf.Tensor: id=48792, shape=(30,), dtype=int64, numpy=
array([0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       1, 0, 0, 0, 0, 0, 0, 0])>

4)  Use tf.math.confusion.matrix in order to determine true positives, true negatives, false positives, and false negatives from the true labels (y_true) and the predictions (y_pred_argmax). 

5) Use seaborn and matplotlib to visualize the confusion matrix. 

In [36]:
%matplotlib inline  
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

def visualize_confusion_matrix(y_pred_argmax, y_true):
    """

    :param y_pred_arg: This is an array with values that are 0 or 1
    :param y_true: This is an array with values that are 0 or 1
    :return:
    """

    cm = tf.math.confusion_matrix(y_true, y_pred_argmax).numpy()
    con_mat_df = pd.DataFrame(cm)

    sns.heatmap(con_mat_df, annot=True, fmt='g', cmap=plt.cm.Blues)
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.show()


visualize_confusion_matrix(y_pred_argmax, y_true)

Analysis of the above shows the following:

* The analysis of the above shows that if the customer is expressing negative sentiment. Then the classifier gets it right 91% of the time. 
* If the model is asked to evaluate positive sentiment, then the model gets it right 66% of the time.


&nbsp;

Congratulations. You made it to the end of the tutorial and now you have a process that takes you from building a data pipeline to having an accurate model.

Also, for the first time in 10 years, I am back on the job market looking for consulting opportunities or full time employment.  If you think I can be of help to you, feel free to reach out. I am on twitter at [@ralphbrooks](https://twitter.com/ralphbrooks) .